In [34]:
pip install transformers


In [35]:
pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

### Import Libraries

In [149]:
import requests
from bs4 import BeautifulSoup


import wordcloud
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import re

#read url
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt


from transformers import pipeline
from transformers import LEDForConditionalGeneration, LEDTokenizer

In [96]:
# Configure pandas to display all text without truncation
pd.set_option('display.max_colwidth', None)


### Gather text from url

In [105]:
#load summarizer

# Load the summarization pipeline
summarizer = pipeline("summarization")
tokenizer = LEDTokenizer.from_pretrained('allenai/led-base-16384')
model = LEDForConditionalGeneration.from_pretrained('allenai/led-base-16384')


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [162]:
def getTextFromURL(url,newsChannel):
  # Fetch the webpage content
  response = requests.get(url)
  #get response
  html = response.content
  return html


# Remove html tags using beautiful soup
def removeHTMLTags(html):
  soup = BeautifulSoup(html, 'html.parser')

  # Decompose the style and script tags
  for data in soup(['style', 'script']):
      data.decompose()

  # Return only the text
  return ' '.join(soup.stripped_strings)

def removeNotNeedeText(text, start, stop):

  #remove text before start
  index = text.find(start)

  if index != -1:
    text = text[index + len(start):]

  #remove text after stop
  index = text.find(stop)

  if index != -1:
    text = text[0:index]

  return text

def removeURL(text):
  # Define a regex pattern for matching URLs
  url_pattern = r'(https?:\/\/(?:www\.)?|www\.)[a-zA-Z0-9-._~:/?#\[\]@!$&\'()*+,;=]+'

  # Use re.sub to replace URLs with an empty string
  cleaned_text = re.sub(url_pattern, '', text)

  return cleaned_text


def getSummarization(text,  max_length_token=16384):

  #calculate the amount of min words and max word the summary should have base on
  # the len of the text, always a 25% of the total len text

  total_words = len(text.split())
  target_summary_length = int(total_words * 0.25)

  # Set min_length and max_length for the summary
  min_length = max(1, target_summary_length - 5)  # Ensure min_length is at least 1
  max_length = target_summary_length + 5

  summary=''
  typePreTrainned = ''
  print(total_words)
  if(total_words<750):
    #use summarizer from hugging face
    result  = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
    summary = result[0]['summary_text']
    typePreTrainned='summarizer'
  else:
    # Tokenize the text and create input tensors
    inputs = tokenizer(text, return_tensors="pt", max_length=max_length_token, truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=1024, min_length=50, length_penalty=2.0, num_beams=4)

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    typePreTrainned='allenai'


    print('sum->',summary)

  return summary, min_length, max_length, typePreTrainned, total_words



def createDFFromURL(url,newsChannel,docType):
  print(url)
  #get txt from html
  html = getTextFromURL(url,newsChannel)

  #remove html tags
  noHTMLText = removeHTMLTags(html)

  #remove urls
  noHTMLText = removeURL(noHTMLText)

  #reduce text base on news source, for cp24 the news end when the word Related Stories appear
  noHTMLText = removeNotNeedeText(noHTMLText, 'Last Updated' , 'Related Stories')
  noHTMLText = removeNotNeedeText(noHTMLText, 'Published' , 'Share:')


  print(noHTMLText)

  summary, min_length, max_length, typePreTrainned, total_words = getSummarization(noHTMLText)

  data = {
      'text' : [noHTMLText],
      'text_length': [len(noHTMLText)],
      'total_word': [total_words],
      'summary': [summary],
      'min_word': [min_length],
      'max_word': [max_length],
      'source': [newsChannel],
      'url' : [url],
      'type': [docType],
      'model_used': [typePreTrainned]
  }

  #create df
  df = pd.DataFrame(data)


  return df


In [161]:
# #generate a list of urls

urlList  = [
    "https://www.cp24.com/news/two-listeriosis-deaths-in-ontario-linked-to-plant-based-milk-recall-moh-1.6968389",
    "https://www.cp24.com/news/flooding-raises-questions-about-toronto-s-vulnerability-to-large-storms-1.6967680",
    "https://www.cp24.com/news/i-actually-rented-a-bike-race-car-driver-almost-misses-honda-indy-press-conference-due-to-toronto-gridlock-1.6969007",
    "https://www.cp24.com/news/grocery-stores-can-sell-ready-to-drink-beverages-as-of-today-as-lcbo-strike-continues-1.6968652",
    "https://www.cp24.com/news/2-vaughan-residents-charged-after-unapproved-prescription-drugs-seized-1.6968686",
    "https://www.cp24.com/news/it-was-like-a-movie-some-toronto-homeowners-facing-flooded-basements-in-wake-of-record-breaking-storm-1.6968798",
    "https://www.cp24.com/news/police-investigation-reports-of-falling-objects-damaging-vehicles-near-the-don-valley-parkway-1.6969051",
    "https://www.cp24.com/news/our-loading-bays-are-flooded-our-freezer-is-broken-north-york-food-bank-appeals-for-help-1.6968547",
    "https://www.cp24.com/news/new-condo-sales-plummeted-to-lowest-level-since-1997-in-first-half-of-year-report-1.6968728",
    "https://www.cp24.com/news/toronto-rapper-s-firearms-charges-dropped-after-police-breached-his-rights-tailored-court-testimonies-judge-1.6968281",
    "https://www.cp24.com/news/police-dismantle-alleged-chop-shop-in-milton-recover-12-stolen-vehicles-1.6968585",
    "https://www.ctvnews.ca/politics/high-profile-liberal-minister-quitting-cabinet-not-running-in-next-election-1.6968645"
    "https://www.cp24.com/news/just-fly-a-drone-over-suggestion-on-encampment-response-at-u-of-t-council-meeting-sparks-privacy-concerns-for-protesters-advocates-1.6968288",
    "https://www.cp24.com/news/more-than-100-residents-rescued-from-mississauga-ont-nursing-home-amid-flooding-1.6968243",
    "https://www.cp24.com/news/ford-to-produce-f-series-pickups-at-oakville-plant-starting-in-2026-1.6968546",
    "https://www.cp24.com/news/partially-treated-wastewater-enough-to-fill-500-olympic-sized-pools-dumped-into-lake-ontario-during-heavy-rainfall-city-1.6967604",
    "https://www.cp24.com/news/former-safety-minister-wants-protective-zones-for-mps-offices-as-threats-increase-1.6968248",
    "https://www.cp24.com/news/flooding-raises-questions-about-toronto-s-vulnerability-to-large-storms-1.6967680",
    "https://www.cp24.com/news/domestic-violence-strategy-should-include-more-prevention-programs-experts-1.6968245",
    "https://www.cp24.com/news/lt-gen-jennie-carignan-to-officially-take-over-command-of-armed-forces-in-ceremony-1.6968251",
    "https://www.cp24.com/news/republican-national-convention-focuses-on-foreign-policy-1.6968265",
    "https://www.cp24.com/news/charges-laid-in-investigation-into-group-allegedly-distributing-narcotics-in-hamilton-niagara-region-1.6968030"




]

# urlList  = [
#     "https://www.cp24.com/news/police-investigation-reports-of-falling-objects-damaging-vehicles-near-the-don-valley-parkway-1.6969051"]

In [163]:


def getFullCSV(urlList, newsChannel, docType):
  #create a global df to save all the scrapped news
  globaldf = pd.DataFrame()

  for url in urlList:
    #get df
    dfwithText = createDFFromURL(url,newsChannel,docType)

    #merge with previous reading
    globaldf = pd.concat([globaldf, dfwithText], ignore_index=True)

    print(globaldf.shape)

  return globaldf





In [164]:

#add all my url news to one final df
df = getFullCSV(urlList,'cp24','news')

#show dimenssions
print(f'shape {df.shape}')


#generate csv
#save news collected in a csv
df.to_csv('news_summarized.csv')


https://www.cp24.com/news/two-listeriosis-deaths-in-ontario-linked-to-plant-based-milk-recall-moh-1.6968389
 Thursday, July 18, 2024 1:56PM EDT Two deaths as a result of a listeriosis outbreak linked to a plant-based milk recall are in Ontario, provincial health officials confirmed Thursday. Silk brand almond milk, coconut milk, almond-coconut milk and oat milk, as well as Great Value brand almond milk, were recalled earlier this month. "At this point in time, the Ministry of Health can confirm that 2 deaths in Ontario have been related to this outbreak," a spokesperson told CP24 in an email. "The Ministry of Health is working closely with Local Public Health Agencies, Public Health Ontario, the CFIA, Health Canada and the Public Health Agency of Canada regarding the recall of these products." At least 12 lab-confirmed cases of listeriosis have been reported in connection with the recall, including 10 in Ontario and one each in Quebec and Nova Scotia. All of the affected products, of w

Input ids are automatically padded from 1407 to 2048 to be a multiple of `config.attention_window`: 1024


 Thursday, July 18, 2024 8:46AM EDT A Toronto rapper's firearm charges have been dropped after a judge found police breached his Charter rights and gave “tailored” testimonies while on the stand. In an Ontario Superior Court decision released last month, Justice Sandra Nishikawa handed Omary Bent, 27, a stay of proceedings. He was acquitted of four firearm charges laid on him in the summer of 2021. The acquittal was granted, in part, after Nishikawa found that the officers used excessive force in dragging Bent out of his Etobicoke condominium by his hair and breached his right to counsel in questioning him without a lawyer present. In turn, the judge found the court should “distance itself” from the evidence obtained during the raid. Download our app to get alerts on your device Get the latest local updates right to your inbox According to court documents, the Toronto Police Service (TPS) began looking into Bent after it was suggested by a confidential source that he’d been dealing dru

Input ids are automatically padded from 1005 to 1024 to be a multiple of `config.attention_window`: 1024


 Thursday, July 18, 2024 4:08PM EDT Gen. Jennie Carignan was a little emotional on Thursday as she spoke to reporters about what it means to take over command of the Canadian Armed Forces. It's an honour that felt like the natural next step in her career, she said, something she's worked toward for the last 38 years. It also means, for other women or marginalized people, that "it's possible, folks — the door is open." "This is actually what my parents used to tell me: 'Anything is possible. You can do anything,'" she said. "But I realized in 1986 that that was not necessarily true for everybody. Even if the doors were open, perceptions stay. There's unconscious biases that, frankly, I do have to question, myself, regularly." Carignan became chief of the defence staff in a ceremony at the Canadian War Museum in Ottawa on Thursday morning, taking over from Gen. Wayne Eyre, who is set to retire after three years in the job. "I feel ready, poised and supported to take on this manifold chal

Input ids are automatically padded from 1038 to 2048 to be a multiple of `config.attention_window`: 1024


 Thursday, July 18, 2024 6:28AM EDT Republicans criticized U.S. President Joe Biden's record on foreign policy at the national convention in Milwaukee, as many party faithful called for America to step back from its contributions to the world. Wednesday's agenda and its theme of "Make America Strong Once Again" will give leaders around the world insight into the potential impact of a second Donald Trump presidency. "No NATO. I want America strong," said Kimberly Nguyen, wearing a star-spangled dress and draped in a Trump 2024 flag outside the convention. Trump, who was officially confirmed as the Republican presidential candidate on Monday, has mused about increased tariffs, been critical of aid to Ukraine and repeatedly claimed he would not defend NATO members that don't meet defence spending targets, of which Canada is one. During a NATO leaders' summit in Washington last week, Prime Minister Justin Trudeau promised to meet the spending target, which is the equivalent of two per cent

In [165]:
df

text  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         Thursday, July 18, 2024 1:56PM EDT Two deaths as a result of a

In [70]:
#see if your data is loaded properly
loaddf = pd.read_csv('news_summarized.csv')

In [71]:
loaddf

Unnamed: 0  \
0           0   
1           1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               